In [ ]:
# 导入所有需要的库
import numpy as np
import time
from pynq import Overlay, allocate
from scipy.io import wavfile
from scipy.signal import firwin
from plotly.offline import init_notebook_mode
import warnings
import plotly.graph_objs as go
import plotly.offline as py
from scipy.signal import spectrogram
from IPython.display import Audio

# 初始化 Notebook 环境以便绘图
init_notebook_mode(connected=True)
warnings.simplefilter(action='ignore', category=FutureWarning)

# --- 1. 加载硬件 Overlay ---
print("正在加载硬件 Overlay...")
overlay = Overlay("./fir.bit")
fir_ip = overlay.fir_wrap_0
print("Overlay 加载成功！")


# --- 2. 准备输入数据 ---
fs, aud_in = wavfile.read("./media/birds.wav")
sample_len = len(aud_in)
print(f"音频数据读取成功，共有 {sample_len} 个采样点。")

nyq = fs / 2.0
taps = 99
hpf_coeffs = firwin(taps, 2800/nyq, pass_zero=False)
hpf_coeffs_quant = hpf_coeffs / np.max(np.abs(hpf_coeffs)) * (2**15 - 1)
hpf_coeffs_hw = np.int32(hpf_coeffs_quant)


# --- 3. 分配可供硬件访问的物理内存 ---
print("正在为 IP 核分配连续物理内存...")
input_buffer = allocate(shape=(sample_len,), dtype='i4')
output_buffer = allocate(shape=(sample_len,), dtype='i4')
coef_buffer = allocate(shape=(taps,), dtype='i4')


# --- 4. 将数据从 Python 复制到物理内存 ---
np.copyto(input_buffer, np.int32(aud_in))
np.copyto(coef_buffer, hpf_coeffs_hw)
print("输入数据和系数已复制到硬件可访问的内存中。")


# --- 5. 配置并启动硬件 IP 核 ---
fir_ip.write(0x1c, input_buffer.physical_address)
fir_ip.write(0x10, output_buffer.physical_address)
fir_ip.write(0x30, coef_buffer.physical_address)
fir_ip.write(0x28, sample_len)
print("IP 核配置完成。")

print("启动硬件加速...")
start_time = time.time()
fir_ip.write(0x00, 0x01)

while fir_ip.read(0x00) & 0x2 == 0:
    pass

end_time = time.time()
print(f"硬件加速完成！")
hw_time = end_time - start_time
print(f"硬件加速耗时: {hw_time:.6f} 秒")


# --- 6. 结果验证与性能对比 ---
sw_time = 0.459
print(f"纯软件滤波耗时: {sw_time:.6f} 秒")
print(f"硬件加速性能提升了约 {sw_time/hw_time:.2f} 倍！")

# 定义绘图函数
def plot_spectrogram(samples, fs, title):
    f_label, t_label, spec_data = spectrogram(samples, fs=fs, mode="magnitude")
    max_heat = np.max(np.abs(samples))
    
    # 创建 Figure 对象
    fig = go.Figure()
    fig.add_trace(go.Heatmap(z=np.clip(spec_data, 0, max_heat*0.5), y=f_label, x=t_label))
    fig.update_layout(title=title, xaxis_title='时间 (s)', yaxis_title='频率 (Hz)')
    
    # 【关键修改处】直接将 Figure 对象传递给 iplot 函数
    py.iplot(fig)

print("\n正在生成硬件滤波后的频谱图...")
plot_spectrogram(output_buffer, fs, title='硬件加速滤波后的频谱图')

# 保存并播放硬件处理后的音频
scaled_output_hw = np.int16(output_buffer / np.max(np.abs(output_buffer)) * 32767)
wavfile.write('hpf_hw.wav', fs, scaled_output_hw)
print("硬件滤波后的音频已保存为 hpf_hw.wav。")
display(Audio('hpf_hw.wav'))